# Instalar Tensorflow

In [ ]:
# 2do paso: importe los siguientes módulos:

import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
from bs4 import BeautifulSoup # para trabajar texto en varios formatos
import re # expresiones regulares

#importar datos


In [ ]:
df = pd.read_csv("SMSSpamCollection(Filtrada).csv", on_bad_lines='skip', sep=';')

In [ ]:
df

#Análisis exploratorio

In [ ]:
#import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#funcion resumen de un dataframe

def resumen(data):
  total_perdidos = data.isnull().sum().sum()
  missing_percent = data.isnull().sum()*100 /len(data)
  total_missing = data.isnull().sum()
  unique = data.nunique()
  not_null = data.notnull().sum()
  df = pd.DataFrame({
      'Valores Unicos' : unique,
      'Valores Existentes': not_null,
      'Total perdidos': total_missing,
      'Porc. perdidos %': missing_percent
  })
  tipos = data.dtypes
  numericas = tipos[tipos != object].index
  no_numericas = tipos[tipos == object].index
  print('1. Las dimensiones de la data:', data.shape)
  print('2. Total Datos perdidos son:', total_perdidos)
  print('3. La cantidad de variables numericas es: ', len(numericas), '\n   Y ellas son:\n', numericas)
  print('4. La cantidad de variables NO numericas es: ', len(no_numericas), '\n  Y ellas son:\n', no_numericas)
  df = df.round(2)
  return(df)

In [ ]:
resumen(df)

In [ ]:
df.groupby('resultado').size()

In [ ]:
sns.countplot(df['resultado'])
print(df['resultado'].value_counts())
plt.title("resultado")
plt.show()

In [ ]:
for i in df:
  print(i, len(df[i].unique()))

Debemos etiquetar a los textos convertidos a vectores

In [ ]:
df['resultado']

#SECCION CON DATOS PROCESADOS

## Preprocesamiento de texto


In [ ]:
df

In [ ]:
df['resultado'] = df['resultado'].apply(lambda x:1 if x == 'ham' else 0) 

In [ ]:
df

## Limpieza de datos

In [ ]:
# El código anterior me trae la última versión de keras
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf

from tensorflow.keras import layers # importar layers desde keras

import tensorflow_datasets as tfds # este me trae los tokens

In [ ]:
def clean_tweet(tweet):

  tweet= BeautifulSoup(tweet,"lxml").get_text() # sirve para trabajar en un formato especial de tratar el texto. hay que usar el get_text para que me lo devuela en un formato legible
  
  # Dejamos todo en minúscula
  tweet = tweet.lower()
  # Reemplazamos puntos por espacios
  tweet = tweet.replace("."," ")
  # Eliminamos la @ y su mención
  tweet = re.sub(r"@[A-Za-z0-9.]+",' ',tweet) # después del @ puede leer lo que se encuentra en el []. El + significa. El r significa raw o string
  # Eliminamos los links de la URLs
  tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet) # aqui el ? me dice que el caracter anterior puede estar o no
  # Nos quedamos solo con los caracteres
  tweet = re.sub(r"[^a-zA-Z.!?']",' ',tweet)# cualquier cosa que no sea lo que viene después de ^ se va a sustituir por espacios en blanco
  # Eliminamos los sitios web
  tweet = re.sub(r"www[A-Za-z0-9./]+",' ', tweet)
  # Eliminamos espacios en blanco adicionales
  tweet = re.sub(r" +",' ',tweet)# si existe más de un espacio en blanco lo reemplazamos por uno solo. El más me dice eso, si hay más de uno reemplazo
  return tweet

In [ ]:
df_clean = [clean_tweet(tweet) for tweet in df.mensaje] # esto me queda en una lista guardado

# Embedding small bert mas Sencillo 128 vectores
https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2

## transformar texto (Embedding)

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df_clean) # df_clean es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

## Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

##SECCION CON DATOS SIN PREPROCESAR

## transformar texto(Embeding) Sin preprocesamiento de texto

In [ ]:
df

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df['mensaje']) # df['mensaje'] es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

### Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

#### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

# Embedding small bert complejo 768 vectores
https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/2

## transformar texto (Embedding)

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/2")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df_clean) # df_clean es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

## Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

#### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

##SECCION CON DATOS SIN PREPROCESAR

## transformar texto(Embeding) Sin preprocesamiento de texto

In [ ]:
df

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/2")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df['mensaje']) # df['mensaje'] es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

### Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

#### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

# Embedding  bert normal L-12 --- 768 Vectores
https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4

## transformar texto (Embedding)

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df_clean) # df_clean es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

## Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

##SECCION CON DATOS SIN PREPROCESAR

## transformar texto(Embeding) Sin preprocesamiento de texto

In [ ]:
df

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df['mensaje']) # df['mensaje'] es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

### Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

#### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

# Embedding  bert normal L-24 --- 1024
https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4

## transformar texto (Embedding)

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df_clean) # df_clean es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

## Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

##SECCION CON DATOS SIN PREPROCESAR

## transformar texto(Embeding) Sin preprocesamiento de texto

In [ ]:
df

In [ ]:
# 4to paso: Vamos a crear dos variables que cargaran en memoria dos funciones para transformar el texto
##
##   - preprocessor: "objeto" que se encargará de transformar el texto en el formato que necesita el modelo BERT para funcionar
##   - encoder: "objeto" que será el modelo BERT y se alimentará del resultado de preprocessor, retornando el texto transformado en un vector numérico.

preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4")

In [ ]:
# 5to paso: Vamos a usar el objeto preprocessor, y guardamos su resultado en la variable embedding
##   A modo de ejemplo para su revisión, solo vamos a transformar dos textos:

embedding = preprocessor(df['mensaje']) # df['mensaje'] es el texto

In [ ]:
# 6to paso: Vamos a codificar el texto con el modelo BERT:
##   - Fijarse que el objeto encoder recibe como argumento la variable embedding (resultado de preprocessor)
##   - Copiar el siguiente código de forma literal

pooled_output = encoder(embedding)["pooled_output"]

In [ ]:
# Opcional: revisamos el contenido de la variable pooled_output, el cual tiene dos vectores de 128 dimensiones
##
##    - Dos vectores, porque le pasamos solo dos textos
##    - 128 dimensiones, porque el modelo BERT utilizado convierte cualquier texto (independiente del largo) en un vector numérico de 128 dimensiones.

pooled_output

In [ ]:
# 7mo paso: El contenido de la variable pooled_output es una estructura de datos de tipo tensorflow.
#           Para transformarlo a un dataframe y usarlo de forma tradicional con los modelos clásicos, 
#           ejecutamos la siguiente linea de código:

dataX2 = pd.DataFrame(pooled_output.numpy())

In [ ]:
# Opcional:  Ahora la variable dataX es un dataframe y podemos revisar su dimensionalidad y visualizarlo:
dataX2.shape

In [ ]:
dataX2

##seleccion de las variable

In [ ]:
X = dataX2 
Y = df['resultado'].copy()

In [ ]:
X.columns

In [ ]:
X.info()

In [ ]:
Y.head()

## Balanceo de datos

### Aplicando Smote

In [ ]:
from imblearn.over_sampling import SMOTE #over sobre nivelar hacia arriba under nivelacion hacia el menor dato
#from imblearn.under_sampling import SMOTE 

smote = SMOTE(random_state = 325, n_jobs=-1)
X_smote, Y_smote = smote.fit_resample(X, Y)# hace Balance de los datos a partir de los agreglos

from collections import Counter

Counter(Y_smote)

## Train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train_smote, X_test_smote, Y_train_smote, Y_test_smote = train_test_split(X_smote, Y_smote, random_state=325,test_size = 0.3)
        
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=325,test_size = 0.3)        

## SVM


###SVC 

#### Caso 1

In [ ]:
parametros = {'C'      : np.arange(1, 5, 1),
            'gamma'  : np.arange(1, 5, 1),
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()

### Nusvm

#### Caso 1

In [ ]:
parametros = {'nu'      : np.arange(0, 1, 0.1),
            'gamma'  : ['scale','auto'],
           'kernel' : ['rbf', 'linear','sigmoid']}

from sklearn.svm import NuSVC
from sklearn.model_selection import GridSearchCV

clf = NuSVC(class_weight='balanced', random_state=325)  
grid_search = GridSearchCV(clf, parametros, n_jobs=-1, cv=5) 

In [ ]:
grid_search.fit(X_train_smote, Y_train_smote)

In [ ]:
print('El mejor modelo:\n', grid_search.best_params_)

In [ ]:
clf_best = grid_search.best_estimator_
pred = clf_best.predict(X_test_smote)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test_smote, pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay.from_estimator(
       clf_best,
       X_test_smote,
       Y_test_smote,
       display_labels=None,
       cmap=plt.cm.Blues,
       normalize=None)

disp.ax_.set_title('Matriz de Confusión')
disp.ax_.set_xlabel('Predicción');
disp.ax_.set_ylabel('Realidad'); 
disp.ax_.xaxis.set_ticklabels(['0', '1']); 
disp.ax_.yaxis.set_ticklabels(['0', '1']);

print('Matriz de Confusión')
print(disp.confusion_matrix)

plt.show()